<a href="https://colab.research.google.com/github/Jaime44/WorkSpace/blob/main/models/colaborative%20filter/00_FC_Recommender_System_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
    GPU = False
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive
    import tensorflow as tf
    print("Versión de TensorFlow:", tf.__version__)
    print("Dispositivos disponibles:", tf.config.list_physical_devices())

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/Models/colaborative filter'

    path_workspace = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)

    # Verificar la GPU
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        print('GPU no encontrada')
    else:
      print(f'Encontrada GPU: {device_name}')
      GPU = True

    # Habilitar la GPU para TensorFlow
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('Memoria de la GPU configurada dinámicamente')
    else:
        print('No se encontraron dispositivos GPU configurables')
else:
    print("El código se está ejecutando en un entorno local.")
    path_workspace ='C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/'
    path_absolute = os.getcwd().replace("\\", "/")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/Models/colaborative filter'

datasets_path = "/datasets/"
path_absolute = path_absolute+datasets_path


sys.path.append(path_workspace)

El código se está ejecutando en un entorno local.


## Se cargan las librerías

In [2]:
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from surprise import Reader, Dataset, SVD, NormalPredictor, BaselineOnly, KNNBasic, NMF
from surprise.model_selection import cross_validate, train_test_split, KFold ,GridSearchCV , RandomizedSearchCV

## METRICS FUNCTIONS

In [3]:
def calculate_f1_at_k(recommended_movies, preferred_movies, k=3):
    """
    Calcula el F1 score en el top-k recomendaciones.

    Args:
    preferred_movies (list): Lista de IDs de películas preferidas por el usuario.
    recommended_movies (list): Lista de IDs de películas recomendadas.
    k (int): Número de recomendaciones top a considerar para el cálculo.

    Returns:
    float: El valor de F1@k.
    """
    # Asegurar que solo se consideren las top-k recomendaciones
    recommended_at_k = recommended_movies[:k]
    
    # preferred_movies = list(relevant_items_df['movieId'])

    if len(preferred_movies) == 0:
        return 0
    
    # Verdaderos positivos: ítems recomendados que el usuario prefiere
    true_positives = len(set(preferred_movies) & set(recommended_at_k))
    
    # Precision@k
    if true_positives > 0:
        precision_at_k = true_positives / len(recommended_at_k)
        recall_at_k = true_positives / len(preferred_movies)
        f1_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)
    else:
        f1_at_k = 0

    return f1_at_k


In [4]:
def recallK(recommended_items, relevant_items, k=3):
    """
    Calcula el Recall en k (Recall@k) o Hit Ratio en k (HitRatio@k) dadas las recomendaciones y los elementos relevantes.

    :param recommended_items: Una lista de los elementos recomendados.
    :param relevant_items: Una lista de los elementos relevantes para el usuario.
    :param k: El número de elementos principales a considerar.
    :return: El Recall en k.
    """
    if k <= 0:
        raise ValueError("k debe ser un entero positivo.")

    # relevant_items = list(relevant_items_df['movieId'])

    if len(relevant_items) == 0:
        return 0

    # Tomamos solo los primeros k elementos recomendados
    recommended_at_k = recommended_items[:k]

    # Contamos el número de elementos relevantes entre los k recomendados
    relevantes_entre_k = sum(1 for item in recommended_at_k if item in relevant_items)

    # Calculamos el Recall en k
    recall_at_k = relevantes_entre_k / len(relevant_items)

    return recall_at_k

In [5]:
def discounted_cumulative_gain(recommended_items, relevant_items):
    dcg = 0
    # print(f'\ndiscounted_cumulative_gain --> recommended_items {recommended_items}')
    # print(f'discounted_cumulative_gain --> relevant_items {relevant_items}\n')
    for i, item in enumerate(recommended_items, start=1):
        if item in relevant_items:
            dcg += 1 / (math.log2(i + 1))
    return dcg

def ideal_discounted_cumulative_gain(recommended_items, relevant_items):
    # print(f'ideal_discounted_cumulative_gain --> recommended_items {recommended_items}')
    # print(f'ideal_discounted_cumulative_gain --> relevant_items {relevant_items}\n')
    sorted_relevant_items = sorted(relevant_items, key=lambda x: recommended_items.index(x) if x in recommended_items else float('inf'))
    return discounted_cumulative_gain(sorted_relevant_items, relevant_items)

def normalized_discounted_cumulative_gain(recommended_items, relevant_items):
    # print(f'\nnormalized_discounted_cumulative_gain --> recommended_items {recommended_items}')
    # print(f'normalized_discounted_cumulative_gain --> relevant_items {relevant_items}')
    dcg = discounted_cumulative_gain(recommended_items, relevant_items)
    idcg = ideal_discounted_cumulative_gain(recommended_items, relevant_items)

    if idcg == 0:
        return 0
    else:
        return round(dcg / idcg, 2)

def mean_normalized_discounted_cumulative_gain(recommended_items, relevant_items):
  # print(f'\nmean_normalized_discounted_cumulative_gain --> recommended_items {recommended_items}')
  # print(f'mean_normalized_discounted_cumulative_gain --> relevant_items {relevant_items}')
  ndcg_values = [normalized_discounted_cumulative_gain(recommended, relevant)
                  for recommended, relevant in zip(recommended_items, relevant_items)]
  average_ndcg = np.mean(ndcg_values)
  return average_ndcg

# Example usage
recommended_items_list = [
    [1, 3, 5, 7, 9],
    [2, 4, 6, 8],
    [11, 12, 13, 14, 15, 16, 17]
]

relevant_items_list = [
    [2, 3, 5, 7, 11],
    [1, 4, 6, 8, 9],
    [16, 17, 18, 19, 20]
]

ndcg_values = [normalized_discounted_cumulative_gain(recommended, relevant)
               for recommended, relevant in zip(recommended_items_list, relevant_items_list)]

print(f"nDCG values: {ndcg_values}")

average_ndcg = mean_normalized_discounted_cumulative_gain(recommended_items_list, relevant_items_list)
print(f"nDCG promedio: {average_ndcg:.2f}")

nDCG values: [0.53, 0.53, 0.23]
nDCG promedio: 0.43


In [6]:
def precisionK(recommended_items, relevant_items, k=3):
    """
    Calcula la precisión en k (Precision@k) dadas las recomendaciones y los elementos relevantes.

    :param recommended_items: Una lista de los elementos recomendados.
    :param relevant_items: Una lista de los elementos relevantes para el usuario.
    :param k: El número de elementos principales a considerar.
    :return: La precisión en k.
    """
    if k <= 0:
        raise ValueError("k debe ser un entero positivo.")

    # relevant_items = list(relevant_items_df['movieId'])

    if len(recommended_items) == 0:
        return 0

    # print(f"-------------------------------> RECOMMEND: \n{recommended_items}")

    # Tomamos solo los primeros k elementos recomendados
    recommended_at_k = recommended_items[:k]

    # print(f"-------------------------------> RECOMMEND: \n{recommended_at_k}")

    # print(f"-------------------------------> RECOMMEND: \n{relevant_items}")

    # Contamos el número de elementos relevantes entre los k recomendados
    relevantes_entre_k = sum(1 for item in recommended_at_k if item in relevant_items)

    # Calculamos la precisión en k
    precision_at_k = relevantes_entre_k / k

    return precision_at_k

In [7]:
def mean_reciprocal_rank(recommended_items_list, relevant_items_list):
    if len(recommended_items_list) != len(relevant_items_list):
        raise ValueError("The length of recommended_items_list and relevant_items_list must be the same.")

    reciprocal_ranks = []

    # Iterate through the lists of recommended items and relevant items for each user
    for recommended_items, relevant_items in zip(recommended_items_list, relevant_items_list):
        # print(f"recommended_items --> {recommended_items}")
        # print(f"relevant_items --> {relevant_items}")
        # Find the reciprocal rank for each user
        for rank, item in enumerate(recommended_items, start=1):
          # print(f"rank --> {rank}")
          # print(f"item --> {item}")
          if item in relevant_items:
              reciprocal_ranks.append(1 / rank)
              break
          else:
              reciprocal_ranks.append(0)

    # Calculate the mean reciprocal rank
    mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)
    return mrr

# # Example usage
# recommended_items_list = [
#     [1, 3, 5, 7, 9],
#     [2, 4, 6, 8],
#     [11, 12, 13, 14, 15, 16, 17]
# ]

# relevant_items_list = [
#     [2, 3, 5, 7, 11],
#     [1, 4, 6, 8, 9],
#     [16, 17, 18, 19, 20]
# ]

# mrr = mean_reciprocal_rank(recommended_items_list, relevant_items_list)
# print(f"Mean Reciprocal Rank: {mrr:.2f}")


In [8]:
def average_precision(recommended_items, relevant_items):
    true_positives = 0
    sum_precisions = 0
    # print(f"AVP_recommended_items --> {recommended_items}")
    # print(f"AVP_relevant_items --> {relevant_items}")
    for rank, item in enumerate(recommended_items, start=1):
      # print(f"AVP_rank --> {rank}")
      # print(f"AVP_item --> {item}")
      if item in relevant_items:
          true_positives += 1
          precision_at_rank = true_positives / rank
          sum_precisions += precision_at_rank
      # print(f"AVP_sum_precisions --> {sum_precisions}")

    return sum_precisions / len(relevant_items) if len(relevant_items) > 0 else 0

def mean_average_precision(recommended_items_list, relevant_items_list):
    if len(recommended_items_list) != len(relevant_items_list):
        raise ValueError("The length of recommended_items_list and relevant_items_list must be the same.")

    average_precisions = []

    # Calculate the average precision for each user
    for recommended_items, relevant_items in zip(recommended_items_list, relevant_items_list):
        ap = average_precision(recommended_items, relevant_items)
        average_precisions.append(ap)

    # Calculate the mean average precision across all users
    map_value = sum(average_precisions) / len(average_precisions)
    return round(map_value, 2)

# Example usage
recommended_items_list = [
    [1, 3, 5, 7, 9],
    [2, 4, 6, 8],
    [11, 12, 13, 14, 15, 16, 17]
]

relevant_items_list = [
    [2, 3, 5, 7, 11],
    [1, 4, 6, 8, 9],
    [16, 17, 18, 19, 20]
]

map_value = mean_average_precision(recommended_items_list, relevant_items_list)
print(f"Mean Average Precision: {map_value}")

Mean Average Precision: 0.29


In [9]:
def catalog_coverage(recommended_items_list, catalog_items):
    # Flatten the list of recommended items and convert it to a set
    unique_recommended_items = set(item for sublist in recommended_items_list for item in sublist)

    # Calculate the intersection of unique recommended items and catalog items
    covered_items = unique_recommended_items.intersection(catalog_items)

    # Calculate the catalog coverage
    coverage = len(covered_items) / len(catalog_items)
    return coverage

# Example usage
recommended_items_list = [
    [1, 3, 5, 7, 9],
    [2, 4, 6, 8],
    [11, 12, 13, 14, 15, 16, 17]
]

catalog_items = set(range(1, 21))

coverage = catalog_coverage(recommended_items_list, catalog_items)
print(f"Catalog Coverage: {coverage}")

Catalog Coverage: 0.8


## Se cargan los Datasets

In [10]:
# dataFrame_onehot_encode = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs_onehot_encode.csv', sep=',')

# dataFrame_onehot_encode.head()


In [11]:
dataFrame_without_ohe = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv')
print(dataFrame_without_ohe.shape)
dataFrame_without_ohe.head()

C:\Users\jaime\AppData\Local\Temp\ipykernel_15172\4218327248.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataFrame_without_ohe = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv')


(25000095, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
0,1,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1147880044,5.0
1,3,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1439474476,5.0
2,4,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1573938898,4.0
3,5,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,830786155,4.0
4,7,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,835444730,4.0


In [12]:
dataFrame_without_ohe_min_ratings = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs_min_ratins.csv')
print(dataFrame_without_ohe_min_ratings.shape)
dataFrame_without_ohe_min_ratings.head()
dataFrame = dataFrame_without_ohe_min_ratings.copy()

(56162, 8)


In [13]:
# dataFrame = dataFrame.sample(n=250000, random_state=42)
df = dataFrame.copy()
print(df.shape)
df.head()

(56162, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
0,521,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1279793342,5.0
1,741,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1262230676,4.0
2,1028,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1303088217,5.0
3,2057,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1432118815,4.5
4,2730,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1450742665,4.0


## Ordenar el dataFrame por la columna 'timestamp' de mayor a menor, de la claifición mas reciente a la más antigua.


In [14]:
dataFrame_sorted_by_timestamp = dataFrame.copy()

# Ordenar el DataFrame por la columna 'timestamp' de mayor a menor
dataFrame_sorted_by_timestamp = dataFrame_sorted_by_timestamp.sort_values(by='timestamp', ascending=False)

## Se eliminan las filas que tengan NaN en las columnas **tag_by_user** o **tag_genome**

In [15]:
def contar_valores_nulos(df):
    # Cuenta los valores nulos en cada columna del DataFrame
    valores_nulos_por_columna = df.isnull().sum()
    # Cuenta los valores nulos en todo el DataFrame
    total_valores_nulos = df.isnull().sum().sum()
    # Imprime la cantidad de valores nulos por columna
    print("Valores nulos por columna:")
    print(valores_nulos_por_columna)
    # Imprime el total de valores nulos en el DataFrame
    print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

def eliminar_filas_nulas(df, columna):
    # Seleccionar las filas con valores nulos en la columna deseada
    filas_con_nulos = df[df[columna].isnull()]
    # Mostrar las filas con valores nulos
    # print("Filas con valores nulos en la columna", columna, ":")
    # print(filas_con_nulos)
    # Eliminar las filas con valores nulos en la columna deseada
    df = df.dropna(subset=[columna])
    return df

In [16]:
# contar_valores_nulos(df)
dataFrame_sorted_by_timestamp = eliminar_filas_nulas(dataFrame_sorted_by_timestamp, 'tag_genome')
# contar_valores_nulos(df)
dataFrame_sorted_by_timestamp = eliminar_filas_nulas(dataFrame_sorted_by_timestamp, 'tag_by_user')
# contar_valores_nulos(df)

In [17]:
print(dataFrame_sorted_by_timestamp.shape)
dataFrame_sorted_by_timestamp.head()

(54671, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
53748,73725,190401,"Scotland, coverup, netflix, dark, Netflix orig...","original, mentor, catastrophe, silly fun, grea...",Calibre (2018),Thriller,1574300676,3.0
30003,122409,1500,"hitman, rekindled love, romance, predictable, ...","comedy, hit men, hitman, off-beat comedy, assa...",Grosse Pointe Blank (1997),Comedy|Crime|Romance,1574276946,2.5
26209,160473,47,"Sloth, s.w.a.t., very good, Atmospheric, drug ...","powerful ending, police investigation, great e...",Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1574238626,4.5
19356,42965,79132,"menswear - outstanding, Orriginal screenplay, ...","complex, dreams, complicated, visually appeali...",Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,1574238008,5.0
38586,96399,2527,"AI, wissenschaftliche sci-fi, cyborgs, chase, ...","futuristic, robot, androids, future, robots",Westworld (1973),Action|Sci-Fi|Thriller|Western,1574219255,2.5


## Se toma un subDataset del original

In [18]:
data = dataFrame_sorted_by_timestamp.copy()
#Tomar una submuestra
# n_samples = round(dataFrame.shape[0] * 0.3)
# data = data.sample(n=n_samples, random_state=42)
df = data.copy()
print(df.shape)
df.head()

(54671, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
53748,73725,190401,"Scotland, coverup, netflix, dark, Netflix orig...","original, mentor, catastrophe, silly fun, grea...",Calibre (2018),Thriller,1574300676,3.0
30003,122409,1500,"hitman, rekindled love, romance, predictable, ...","comedy, hit men, hitman, off-beat comedy, assa...",Grosse Pointe Blank (1997),Comedy|Crime|Romance,1574276946,2.5
26209,160473,47,"Sloth, s.w.a.t., very good, Atmospheric, drug ...","powerful ending, police investigation, great e...",Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1574238626,4.5
19356,42965,79132,"menswear - outstanding, Orriginal screenplay, ...","complex, dreams, complicated, visually appeali...",Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,1574238008,5.0
38586,96399,2527,"AI, wissenschaftliche sci-fi, cyborgs, chase, ...","futuristic, robot, androids, future, robots",Westworld (1973),Action|Sci-Fi|Thriller|Western,1574219255,2.5


In [19]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# FILTRADO COLABORATIVO BASADO EN MODELOS

Sí, el filtrado colaborativo basado en modelos abarca varios enfoques y técnicas que se utilizan para predecir las preferencias de los usuarios sobre los ítems. A diferencia del filtrado colaborativo basado en memoria, que utiliza directamente los datos de calificación para encontrar similitudes entre usuarios o ítems, el filtrado basado en modelos implica la construcción de modelos predictivos para estimar las calificaciones. Estos son algunos de los enfoques más comunes dentro del filtrado colaborativo basado en modelos:

### 1. **Factorización de Matrices**
Es uno de los enfoques más populares y efectivos para el filtrado colaborativo basado en modelos. La idea es descomponer la matriz de utilidad original (usuarios x ítems) en dos matrices de factores latentes más pequeñas, una que representa las características latentes de los usuarios y otra que representa las características latentes de los ítems. La factorización de matrices más conocida en este contexto es la Descomposición en Valores Singulares (SVD, por sus siglas en inglés).

### 2. **Modelos de Aprendizaje Automático Tradicionales**
Se pueden aplicar modelos de regresión, clasificación o incluso redes neuronales para predecir las calificaciones de los usuarios a los ítems. En este enfoque, cada calificación se trata como una instancia de entrenamiento, donde las características pueden incluir información sobre el usuario, el ítem, o ambos.

### 3. **Redes Neuronales y Deep Learning**
Con el auge del aprendizaje profundo, se han desarrollado modelos más avanzados y complejos, como las redes neuronales profundas (DNN), las redes neuronales convolucionales (CNN) para datos estructurados y las redes neuronales recurrentes (RNN) para secuencias de datos. Estos modelos pueden capturar relaciones no lineales complejas y características latentes a partir de grandes volúmenes de datos.

### 4. **Modelos Basados en Vecinos con Aprendizaje**
Aunque conceptualmente similar al filtrado colaborativo basado en memoria, este enfoque utiliza técnicas de aprendizaje automático para optimizar cómo se calculan y combinan las similitudes entre usuarios o ítems, permitiendo una mejor personalización y precisión.

### 5. **Modelos Híbridos**
Combinan varios de los enfoques mencionados anteriormente para aprovechar sus respectivas fortalezas. Por ejemplo, un modelo híbrido podría combinar factorización de matrices con características adicionales de los usuarios y los ítems en un modelo de aprendizaje profundo para mejorar las predicciones de las calificaciones.

Cada uno de estos enfoques tiene sus propias ventajas y desventajas, y la elección entre ellos depende de varios factores, incluyendo la naturaleza y el tamaño del conjunto de datos, la disponibilidad de información adicional sobre usuarios e ítems, y los objetivos específicos del sistema de recomendación.

El siguiente código se está haciendo una recomendación de películas utilizando el filtrado colaborativo mediante el algoritmo de Descomposición de Valores Singulares (SVD, por sus siglas en inglés). El filtrado colaborativo es un método para hacer recomendaciones automáticas basadas en los gustos y preferencias de otros usuarios. Aquí se describe brevemente el proceso realizado en el código:

1. **Preparación de Datos:** Se utiliza la biblioteca `Surprise` para manejar los datos de calificaciones de usuarios, especificando el rango de calificaciones posibles con el objeto `Reader` y cargando el DataFrame de calificaciones al formato que `Surprise` puede procesar.

2. **Entrenamiento y Validación del Modelo:** El modelo SVD es inicializado y validado usando validación cruzada con las métricas RMSE (Raíz del Error Cuadrático Medio) y MAE (Error Absoluto Medio) para evaluar su rendimiento antes y después del entrenamiento.

3. **Entrenamiento del Modelo SVD:** Se entrena el modelo SVD con el conjunto de entrenamiento derivado de los datos. Este modelo aprende a predecir la calificación que un usuario daría a una película basada en las calificaciones de otros usuarios a esa misma película y a otras películas.

4. **Recomendación de Películas:** Se identifican las películas que el usuario no ha visto y se utiliza el modelo SVD para predecir la calificación que el usuario daría a estas películas no vistas. Luego, se seleccionan las `n_recommendations` películas con las calificaciones estimadas más altas.

5. **Resultado:** Se retorna un DataFrame que contiene las películas recomendadas junto con la calificación predicha para cada una, ordenadas de mayor a menor por la calificación estimada.

Por lo tanto, sí, se está utilizando el filtrado colaborativo basado en modelo (específicamente SVD) para hacer recomendaciones personalizadas de películas a un usuario.

El método descrito en el código utiliza el filtrado colaborativo **basado en modelo**, específicamente a través del uso del algoritmo de Descomposición de Valores Singulares (SVD, por sus siglas en inglés). 

En los sistemas de recomendación, el filtrado colaborativo puede clasificarse principalmente en dos tipos:

1. **Filtrado colaborativo basado en memoria:** Este enfoque hace recomendaciones basadas en las calificaciones previas de todos los usuarios sin asumir un modelo específico. Utiliza técnicas como la similitud coseno o la correlación de Pearson para encontrar usuarios "similares" o ítems "similares" y hacer recomendaciones. Este enfoque puede ser más intuitivo pero puede sufrir en términos de escalabilidad y rendimiento con conjuntos de datos muy grandes.

2. **Filtrado colaborativo basado en modelo:** Este enfoque construye un modelo a partir de los datos de calificaciones de los usuarios para predecir calificaciones desconocidas. El modelo puede ser de factorización matricial, como SVD, o utilizar otras técnicas de aprendizaje automático. Este método tiende a ser más escalable y puede manejar mejor conjuntos de datos grandes y esparsos, ofreciendo también la posibilidad de capturar patrones más complejos en los datos.

El uso de SVD en el código proporciona un ejemplo de filtrado colaborativo basado en modelo, ya que implica la construcción de un modelo matemático (descomposición de la matriz de calificaciones) para predecir las calificaciones de los ítems no calificados por un usuario y hacer recomendaciones en base a esas predicciones.

In [20]:
from surprise.model_selection import train_test_split

In [21]:
def get_trained_svd_model_trained(dataF):
    # Inicialización del modelo SVD
    svd = SVD()
    # Entrenamiento del modelo en el conjunto de entrenamiento
    svd.fit(dataF)
    return svd

In [22]:
def get_train_test_dataset(dataF):
    # Interpretar el conjunto de datos de calificaciones. Se especifica el rango de 
    # las calificaciones en el conjunto de datos, indicando que las calificaciones 
    # pueden variar entre 0.5 y 5. 
    reader = Reader(rating_scale=(0.5, 5))

    # Carga el conjunto de datos desde un DataFrame en un formato que Surprise 
    # puede procesarreader es el argumento que se pasa para especificar cómo se
    # deben interpretar las calificaciones en el conjunto de datos,
    data = Dataset.load_from_df(dataF[['userId', 'movieId', 'rating']], reader)

    return data

In [23]:
def svd_hyperparameter_search(dataF, param_grid, cv):
    gs = GridSearchCV(SVD, param_grid, measures=['RMSE', 'MAE'], cv=cv)
    gs.fit(dataF)
    return gs.best_params, gs.best_score, gs.cv_results, gs.best_estimator

In [24]:
def get_resultls_cross_validation(svd, dataF):
    results_cv = cross_validate(svd, dataF, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    results_cv

In [25]:
def get_relevants_items_by_user(user_id, dataF, threshold=3.5):
    filtered_df = dataF[(dataF['userId'] == user_id) & (dataF['rating'] >= threshold)].sort_values(by='rating', ascending=False)
    
    # Extrae los IDs de las películas del DataFrame filtrado y ordenado.
    # preferred_movies = filtered_df['movieId'].tolist()
    
    return filtered_df

In [26]:
def recommend_movies(svd_model, user_id, dataF):
    # Obtener películas calificadas por el usuario
    rated_movies = dataF[dataF['userId'] == user_id]['movieId'].unique()
    
    # Generar predicciones de rating para las películas calificadas
    predicciones = [(user_id, movie_id, svd_model.predict(user_id, movie_id).est) for movie_id in rated_movies]
    
    # Ordenar las predicciones por rating estimado de mayor a menor
    recommendations = sorted(predicciones, key=lambda x: x[2], reverse=True)
    
    # Preparar lista para el DataFrame final
    recommended_movies_info = []
    for user_id, movie_id, predicted_rating in recommendations:
        title = dataF[dataF['movieId'] == movie_id]['title'].iloc[0]
        recommended_movies_info.append((user_id, movie_id, title, predicted_rating))
    
    # Crear un DataFrame a partir de las recomendaciones
    df_recommended_movies = pd.DataFrame(recommended_movies_info, columns=['userId', 'movieId', 'title', 'predicted_rating'])
    
    return df_recommended_movies


In [27]:
data_reader = get_train_test_dataset(df_train)

In [28]:
cv = 10
param_grid = {'n_epochs': [5, 10, 50, 100], 
              'lr_all': [0.0001, 0.0005, 0.001, 0.01], 
              "reg_all": [0.2, 0.4, 0.6]}

best_params, best_score, cv_results, best_estimator = svd_hyperparameter_search(data_reader, param_grid, cv)

print(f"Mejores parametros --> {best_params}")
print(f"Mejores resultados --> {best_params}")

mejor_modelo_rmse = best_estimator['rmse']
mejor_modelo_mae = best_estimator['mae']

Mejores parametros --> {'rmse': {'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.2}, 'mae': {'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.2}}
Mejores resultados --> {'rmse': {'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.2}, 'mae': {'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.2}}


In [29]:
get_resultls_cross_validation(mejor_modelo_rmse, data_reader)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8941  0.8685  0.8879  0.8950  0.8892  0.8869  0.0096  
MAE (testset)     0.6513  0.6430  0.6480  0.6557  0.6482  0.6493  0.0042  
Fit time          1.31    1.41    1.49    1.46    1.62    1.46    0.10    
Test time         0.05    0.05    0.07    0.08    0.16    0.08    0.04    


In [30]:
get_resultls_cross_validation(mejor_modelo_mae, data_reader)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8793  0.8871  0.8960  0.9012  0.8756  0.8878  0.0097  
MAE (testset)     0.6439  0.6510  0.6580  0.6549  0.6418  0.6499  0.0062  
Fit time          1.50    1.41    1.31    1.37    1.33    1.38    0.07    
Test time         0.05    0.10    0.05    0.06    0.06    0.06    0.02    


Al comparar los resultados de la validación cruzada para ambos modelos (mejor_modelo_rmse y mejor_modelo_mae), vemos que los promedios de RMSE y MAE son muy cercanos entre sí, lo que indica un rendimiento similar en términos de precisión de las predicciones. Sin embargo, hay algunas diferencias en sus métricas que podrían influir en la decisión de cuál elegir:

1. **RMSE (Root Mean Square Error)**:
   - El RMSE promedio para ambos modelos es idéntico (0.8816), lo cual indica que, en promedio, ambos modelos tienen una precisión similar al predecir las calificaciones de las películas.
   - La desviación estándar (Std) en los RMSE es más baja en el modelo optimizado para RMSE (0.0076) en comparación con el modelo optimizado para MAE (0.0209), lo que sugiere que el modelo RMSE tiene una consistencia ligeramente mejor en diferentes conjuntos de datos.

2. **MAE (Mean Absolute Error)**:
   - Los MAE promedios son también muy similares entre los dos modelos, con una ligera ventaja para el modelo MAE (0.6443 vs. 0.6444), aunque esta diferencia es mínima.
   - La desviación estándar en los MAE es mayor en el modelo optimizado para MAE, lo que indica una variabilidad ligeramente mayor en su rendimiento a través de diferentes conjuntos de datos.

3. **Tiempo de Ajuste (Fit time)** y **Tiempo de Prueba (Test time)**:
   - Los tiempos de ajuste y prueba son comparables entre los dos modelos, con una ligera ventaja en rapidez para el modelo RMSE en el tiempo de ajuste.

### Elección del Modelo:
Considerando estas observaciones, **el modelo optimizado para RMSE** podría ser ligeramente preferible debido a:
- Su consistencia más alta (menor desviación estándar en RMSE).
- Tiempo de ajuste levemente más rápido, lo que puede ser beneficioso si se trabaja con conjuntos de datos más grandes o se requiere reajustar el modelo con frecuencia.

### Consideraciones Adicionales:
- Con RMSE el objetivo principal es minimizar los errores grandes en las predicciones (dando más peso a estos errores).
- Con MAE se trata a todos los errores de manera uniforme (sin dar demasiado peso a los errores extremadamente grandes).

### Conclusión:
Basándome en los resultados presentados y asumiendo que la consistencia y precisión en las predicciones son prioritarias, **elegiría el modelo optimizado para RMSE**. 

In [31]:
model = mejor_modelo_rmse

In [32]:
user_id_random = random.choice(df['userId'])
user_id_random

48627

In [62]:
len(df['userId'].unique())

6067

In [33]:
df_test_aux = df_test.copy()
df_test_aux = df_test[df_test['userId'] == user_id_random].sort_values(by='rating', ascending=False)
movies_rated_by_user = df_test_aux.copy()

In [34]:
preferred_movies = get_relevants_items_by_user(user_id_random, df_test)
preferred_movies_aux = preferred_movies.copy()
list_items_relevants_to_user = list(preferred_movies_aux['movieId'])

In [35]:
predicitons = recommend_movies(model, user_id_random, df_test)
list_predicitons = list(predicitons['movieId'])

In [36]:
precision_value_k_point = precisionK(list_predicitons, list_items_relevants_to_user, 5)
precision_value_k_point

0.6

In [37]:
recall_value_k_point = recallK(list_predicitons, list_items_relevants_to_user, 5)
recall_value_k_point

0.06

In [38]:
f1_value_k_point = calculate_f1_at_k(list_predicitons, list_items_relevants_to_user, 5)
f1_value_k_point

0.1090909090909091

In [39]:

list_movies_rated_by_user = list(movies_rated_by_user)
mrr = mean_reciprocal_rank([list_items_relevants_to_user], [list_predicitons])
mrr

1.0

In [40]:
def obtain_precission_recall_f1_by_user(model, user_id_random, df_test):

   predicitons = recommend_movies(model, user_id_random, df_test)
   list_predicitons = list(predicitons['movieId'])

   preferred_movies = get_relevants_items_by_user(user_id_random, df_test)
   preferred_movies_aux = preferred_movies.copy()
   list_items_relevants_to_user = list(preferred_movies_aux['movieId'])
   
   
   precision_value_k_point = precisionK(list_predicitons, list_items_relevants_to_user, 5)

   recall_value_k_point = recallK(list_predicitons, list_items_relevants_to_user, 5)

   f1_value_k_point = calculate_f1_at_k(list_predicitons, list_items_relevants_to_user, 5)
   

   # print(f"Precisión@K: {precision_value_k_point:.2f}")
   # print(f"Recall@K: {recall_value_k_point:.2f}\n")
   # print(f"F1@K: {f1_value_k_point:.2f}")
   
   return precision_value_k_point, recall_value_k_point, f1_value_k_point, list_predicitons, list_items_relevants_to_user


In [41]:
# Obtener la lista de usuarios únicos
lista_usuarios_unicos = df_test['userId'].unique()
# lista_usuarios_unicos = [14208]
num_user_uq = len(lista_usuarios_unicos)
# Iteramos sobre las calificaciones
list_recomendeds = []
list_relevants = []

for unique_user in lista_usuarios_unicos:
  precsionk, recallk, f1k, recommended_items_list, relevant_items_list = obtain_precission_recall_f1_by_user(model, unique_user, df_test)

  list_recomendeds.append(recommended_items_list)
  list_relevants.append(relevant_items_list)

mrr = mean_reciprocal_rank(list_recomendeds, list_relevants)
map_value = mean_average_precision(list_recomendeds, list_relevants)
ndcg_avg = mean_normalized_discounted_cumulative_gain(list_recomendeds, list_relevants)
catalog_items = set(df_test['movieId'])
coverage = catalog_coverage(list_recomendeds, catalog_items)

print(f"MRR: {mrr:.2f}\n")
print(f"MAP: {map_value:.2f}\n")
print(f"nDGC Average: {ndcg_avg:.2f}\n")
print(f"Catalog Coverage: {coverage}")

MRR: 0.84

MAP: 0.89

nDGC Average: 0.90

Catalog Coverage: 1.0


In [42]:
len(lista_usuarios_unicos)

3585

El enfo anterior, que utiliza el algoritmo SVD (Descomposición en Valores Singulares) para generar recomendaciones, encaja en el enfoque de **Factorización de Matrices** dentro del filtrado colaborativo basado en modelos. Este es el motivo:

### ¿Por qué encaja en Factorización de Matrices?

1. **Descomposición de la Matriz de Utilidad**: El uso del algoritmo SVD es una técnica clásica de factorización de matrices que descompone la matriz original de utilidad (calificaciones de usuarios por ítems) en componentes latentes. Estos componentes latentes representan características subyacentes tanto de los usuarios como de los ítems, que no son directamente observables pero que influyen en cómo los usuarios califican los ítems.

2. **Predicción de Calificaciones**: A través de la factorización, SVD reduce la dimensionalidad de la matriz de utilidad, lo que permite predecir calificaciones faltantes en la matriz original. Estas predicciones se basan en las interacciones latentes entre usuarios e ítems, lo que permite hacer recomendaciones personalizadas incluso para combinaciones de usuario-ítem que no aparecen en el conjunto de datos original.

3. **Basado en Modelos**: A diferencia del filtrado colaborativo basado en memoria, que directamente utiliza las calificaciones existentes para encontrar similitudes, el enfoque basado en SVD construye un modelo matemático para explicar y predecir las calificaciones. Este modelo se entrena con los datos existentes y luego se utiliza para hacer predicciones sobre datos no vistos.

### Ventajas de SVD en el Contexto de Recomendaciones:

- **Eficiencia y Escalabilidad**: Al reducir la dimensionalidad de la matriz de utilidad, SVD puede manejar grandes conjuntos de datos más eficientemente que los enfoques basados en memoria.

- **Manejo de Datos Faltantes**: SVD es capaz de predecir calificaciones faltantes, lo que es especialmente útil en sistemas de recomendación donde es común tener matrices de utilidad muy dispersas.

- **Descubrimiento de Factores Latentes**: Al identificar patrones y características subyacentes en las interacciones de los usuarios con los ítems, SVD puede revelar insights valiosos que no son inmediatamente obvios.

En resumen, el uso de SVD para hacer recomendaciones es un claro ejemplo de un enfoque de filtrado colaborativo basado en modelos, específicamente dentro del paradigma de factorización de matrices, debido a cómo trata de inferir las preferencias de los usuarios y las características de los ítems a través de la construcción y aplicación de un modelo matemático.

--------------------------

# FILTRADO COLABORATIVO BASADO EN MEMORIA

### Creación y Preparación de la Matriz de Utilidad para Sistemas de Recomendación

La `ratings_matrix` es una matriz de utilidad que representa las calificaciones que los usuarios han dado a diferentes ítems (por ejemplo, películas, libros, productos, etc.) en un sistema de recomendación. En el contexto de las películas, las filas de esta matriz representan a los usuarios, las columnas representan a las películas, y cada entrada en la matriz contiene la calificación que un usuario ha dado a una película. Si un usuario no ha calificado una película, la entrada correspondiente suele estar vacía o contiene un valor específico que indica falta de calificación (como NaN en pandas).

Para obtener la `ratings_matrix` a partir de un conjunto de datos que incluye calificaciones de usuarios, se puede utilizar la biblioteca pandas en Python de la siguiente manera:

1. Supongamos que tienes un DataFrame llamado `ratings` que incluye al menos tres columnas: `userId`, `movieId` y `rating`. Este DataFrame representa las calificaciones individuales que cada usuario ha dado a las películas.

2. Puedes transformar este DataFrame en una matriz de utilidad (ratings_matrix) usando el método `pivot_table` de pandas. Aquí hay un ejemplo de cómo hacerlo:

```python
import pandas as pd

# Supongamos que ratings es tu DataFrame y tiene las columnas 'userId', 'movieId' y 'rating'
# Ejemplo de cómo podría lucir tu DataFrame
#   userId  movieId  rating
# 0      1        2     3.5
# 1      1       29     3.5
# 2      2        2     3.5

# Convertir el DataFrame de calificaciones en una matriz de utilidad
ratings_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Llenar los valores faltantes con NaN o con otro valor que indique que no hay calificación
ratings_matrix.fillna(0, inplace=True)  # Aquí se rellenan con 0 las películas no calificadas

print(ratings_matrix.head())
```

En el resultado, cada fila corresponde a un `userId`, cada columna a un `movieId`, y las celdas contienen las calificaciones dadas por los usuarios a las películas. Si se utiliza `fillna(0)`, las películas no calificadas por un usuario se rellenan con 0. Alternativamente, se pueden dejar como `NaN` si se prefiere indicar explícitamente la ausencia de calificación.

Esta matriz es esencial para muchos sistemas de recomendación, especialmente para aquellos que utilizan filtrado colaborativo, ya que permite identificar fácilmente las preferencias de los usuarios y encontrar similitudes entre ellos o entre ítems.

In [43]:
# Crea una matriz de utilidad de usuario-película a partir de las calificaciones, 
# con usuarios en filas y películas en columnas. Los valores faltantes (NaN) 
# se rellenan con ceros, indicando ausencia de calificación. La matriz resultante 
# facilita la implementación de algoritmos de recomendación, mostrando las 
# dimensiones de la matriz y un vistazo a las primeras filas para inspección. 
# Este proceso es esencial en el filtrado colaborativo para entender las preferencias 
# de los usuarios y predecir calificaciones futuras.
pivot_table_result_based_user = df.pivot_table(index='userId', columns='title', values='rating')
pivot_table_result_based_user.fillna(0, inplace=True)
pivot_table_result_based_user.shape
pivot_table_result_based_user.head()

title,'71 (2014),'Salem's Lot (2004),"'burbs, The (1989)",(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",100 Girls (2000),"1000 Eyes of Dr. Mabuse, The (Die 1000 Augen des Dr. Mabuse) (1960)",...,Zoolander (2001),Zootopia (2016),Zorba the Greek (Alexis Zorbas) (1964),[REC] (2007),[REC]² (2009),eXistenZ (1999),iBoy (2017),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Cuando usas `pivot_table` para crear una matriz de utilidad a partir de un conjunto de datos de calificaciones, y luego utilizas esta matriz directamente para hacer recomendaciones (por ejemplo, calculando similitudes entre usuarios o ítems), estás empleando un enfoque de **filtrado colaborativo basado en memoria**.

El **filtrado colaborativo basado en memoria** se divide en dos enfoques principales:

1. **Basado en Usuarios**: Se calculan las similitudes entre usuarios basándose en sus patrones de calificación. Para un usuario dado, el sistema encuentra otros usuarios similares y recomienda ítems que estos usuarios similares han calificado positivamente pero que el usuario objetivo no ha visto aún.

2. **Basado en Ítems**: Se calculan las similitudes entre ítems basándose en cómo los usuarios han calificado estos ítems. Si un usuario calificó positivamente un ítem, el sistema recomienda otros ítems similares a este.

Al utilizar `pivot_table` para organizar las calificaciones de los usuarios en una matriz donde las filas representan usuarios y las columnas representan ítems (en este caso, títulos de películas), y luego rellenar los valores faltantes con 0, estás preparando los datos para aplicar este tipo de filtrado colaborativo. Dependiendo de cómo procedas a hacer las recomendaciones (calculando similitudes entre filas para usuarios o entre columnas para ítems), estarás aplicando uno de los dos enfoques de filtrado colaborativo basado en memoria mencionados anteriormente.

Este enfoque es directo y puede ser muy efectivo, especialmente en sistemas con un número moderado de usuarios e ítems. Sin embargo, puede enfrentar desafíos de escalabilidad y rendimiento a medida que el número de usuarios e ítems crece, además de tener que manejar la matriz de utilidad que puede ser muy dispersa (llena de ceros en muchas entradas debido a la falta de calificaciones).

In [44]:
# Se implementa el recomendador según una pelicula. Se usa la función de correlación.

def recomendar_peliculas(nombre_pelicula, tabla_pivote, tipo_correlacion='pearson'):
    """
    Recomienda películas basadas en la correlación de calificaciones de usuarios.

    Parámetros:
    - nombre_pelicula: Nombre de la película para la cual se desean recomendaciones.
    - tabla_pivote: DataFrame que contiene las calificaciones de los usuarios (formato de tabla pivote).
    - tipo_correlacion: Tipo de correlación a utilizar (pearson, kendall, spearman).

    Retorna:
    - DataFrame de películas recomendadas con puntajes de correlación.
    """

    # Extraer las calificaciones de usuarios para la película especificada
    calificaciones_usuario = tabla_pivote[nombre_pelicula]

    # Calcular correlaciones con otras películas.
    # Usando correlación de Pearson default
    # Compara las calificaciones de la película de entrada con las
    # calificaciones de cada otra película en términos de su similitud
    peliculas_similares = tabla_pivote.corrwith(calificaciones_usuario, method=tipo_correlacion)

    # Crear un DataFrame con los puntajes de correlación
    peliculas_corr = pd.DataFrame(peliculas_similares, columns=['Correlacion'])

    # Eliminar películas sin datos de correlación
    peliculas_corr.dropna(inplace=True)

    # Filtrar películas según la cantidad mínima de calificaciones y ordenar por correlación
    peliculas_recomendadas = peliculas_corr.sort_values('Correlacion', ascending=False)

    return peliculas_recomendadas


In [45]:
# Ejemplo usando correlación de Pearson
recomendaciones_pearson_ptr = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_result_based_user, tipo_correlacion='pearson')

# Ejemplo usando correlación de Kendall
recomendaciones_kendall_ptr = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_result_based_user, tipo_correlacion='kendall')

# Ejemplo usando correlación de Spearman
recomendaciones_spearman_ptr = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_result_based_user, tipo_correlacion='spearman')


In [46]:
recomendaciones_pearson_ptr.head()

,Correlacion
title,
Harry Potter and the Goblet of Fire (2005),1.000000
Harry Potter and the Prisoner of Azkaban (2004),0.598181
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.554262
Harry Potter and the Chamber of Secrets (2002),0.551057
Harry Potter and the Order of the Phoenix (2007),0.507605


In [47]:
recomendaciones_kendall_ptr.head()

,Correlacion
title,
Harry Potter and the Goblet of Fire (2005),1.000000
Harry Potter and the Prisoner of Azkaban (2004),0.560622
Harry Potter and the Chamber of Secrets (2002),0.532011
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.496623
Harry Potter and the Order of the Phoenix (2007),0.494801


In [48]:
recomendaciones_spearman_ptr.head()

,Correlacion
title,
Harry Potter and the Goblet of Fire (2005),1.000000
Harry Potter and the Prisoner of Azkaban (2004),0.561389
Harry Potter and the Chamber of Secrets (2002),0.532771
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.497655
Harry Potter and the Order of the Phoenix (2007),0.495408


In [49]:
def recomendar_peliculas(usuario_id, tabla_pivote,
                         top_n=10, tipo_correlacion='pearson'):
    """
    Recomienda películas no vistas por un usuario basándose en la correlación de calificaciones de usuarios.

    Parámetros:
    - usuario_id: ID del usuario para el cual se desean recomendaciones.
    - tabla_pivote: DataFrame que contiene las calificaciones de los usuarios (formato de tabla pivote).
    - top_n: Número de películas recomendadas a devolver.
    - tipo_correlacion: Tipo de correlación a utilizar (pearson, kendall, spearman).

    Retorna:
    - DataFrame de películas no vistas recomendadas con puntajes de correlación.
    """

    # Extraer las calificaciones del usuario
    calificaciones_usuario = tabla_pivote.loc[usuario_id]

    # Filtrar películas no vistas por el usuario
    peliculas_no_vistas = calificaciones_usuario[calificaciones_usuario.isna()].index

    # Calcular correlaciones con las películas no vistas. Usando el tipo de correlación especificado
    # Mide la correlación lineal entre dos variables continuas. Toma valores entre -1 y 1, donde 1
    # significa una correlación positiva perfecta, -1 significa una correlación negativa perfecta, y
    # 0 significa ausencia de correlación lineal.
    peliculas_similares = tabla_pivote[peliculas_no_vistas].corrwith(calificaciones_usuario, method=tipo_correlacion)

    # Crear un DataFrame con los puntajes de correlación
    peliculas_corr = pd.DataFrame(peliculas_similares, columns=['Correlacion'])

    # Eliminar películas sin datos de correlación
    peliculas_corr.dropna(inplace=True)

    # Filtrar películas según la cantidad mínima de calificaciones y ordenar por correlación
    peliculas_recomendadas = peliculas_corr.sort_values('Correlacion', ascending=False)

    # Tomar las primeras 'top_n' películas recomendadas
    peliculas_recomendadas = peliculas_recomendadas.head(top_n)

    return peliculas_recomendadas


In [50]:
userID = dataFrame['userId'].sample().iloc[0]

# Ejemplo usando correlación de Pearson
recomendaciones_pearson = recomendar_peliculas(userID, pivot_table_result_based_user,
                                               top_n=10, tipo_correlacion='pearson')

print(f"Peliculas recomendadas para el usuario {userID}. pearson: ")
recomendaciones_pearson.head(10)

# Ejemplo usando correlación de Kendall
recomendaciones_kendall = recomendar_peliculas(userID, pivot_table_result_based_user, 
                                               top_n=10, tipo_correlacion='kendall')

print(f"Peliculas recomendadas para el usuario {userID}. kendall: ")
recomendaciones_kendall.head(10)

# Ejemplo usando correlación de Spearman
recomendaciones_spearman = recomendar_peliculas(userID, pivot_table_result_based_user,
                                                top_n=10, tipo_correlacion='spearman')

print(f"Peliculas recomendadas para el usuario {userID}. spearman: ")
recomendaciones_spearman.head(10)


Peliculas recomendadas para el usuario 113690. pearson: 
Peliculas recomendadas para el usuario 113690. kendall: 
Peliculas recomendadas para el usuario 113690. spearman: 


,Correlacion
title,


In [51]:
# RESET dataframe variable
df = dataFrame_sorted_by_timestamp.copy()
df.head()

,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
53748,73725,190401,"Scotland, coverup, netflix, dark, Netflix orig...","original, mentor, catastrophe, silly fun, grea...",Calibre (2018),Thriller,1574300676,3.0
30003,122409,1500,"hitman, rekindled love, romance, predictable, ...","comedy, hit men, hitman, off-beat comedy, assa...",Grosse Pointe Blank (1997),Comedy|Crime|Romance,1574276946,2.5
26209,160473,47,"Sloth, s.w.a.t., very good, Atmospheric, drug ...","powerful ending, police investigation, great e...",Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1574238626,4.5
19356,42965,79132,"menswear - outstanding, Orriginal screenplay, ...","complex, dreams, complicated, visually appeali...",Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,1574238008,5.0
38586,96399,2527,"AI, wissenschaftliche sci-fi, cyborgs, chase, ...","futuristic, robot, androids, future, robots",Westworld (1973),Action|Sci-Fi|Thriller|Western,1574219255,2.5


**Cada uno de estos algoritmos sirve a un propósito diferente dentro del espectro del filtrado colaborativo, desde proporcionar un punto de referencia básico (NormalPredictor), pasando por la extracción de factores latentes a través de la factorización de matrices (NMF), hasta el cálculo de similitudes directas en un enfoque basado en memoria (KNNBasic).**

In [52]:
# La línea de código carga un conjunto de datos utilizando la librería surprise
# para modelos de recomendación. Se seleccionan las columnas 'userId', 'movieId', y
# 'rating' del DataFrame ratings. El objeto Reader() se utiliza para especificar cómo
# interpretar las calificaciones, y luego Dataset.load_from_df carga los datos en un
# formato compatible con surprise. Este conjunto de datos surprise puede ser empleado para
# entrenar y evaluar modelos de recomendación basados en filtrado colaborativo.
# Al incluir el argumento rating_scale=(0.5, 5) al crear el objeto Reader,
# le estás diciendo a surprise que las calificaciones están en el rango de 0.5 a 5
reader = Reader(rating_scale=(0.5, 5))
sup_data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)


## 1. **NormalPredictor**
- **Enfoque**: Aleatorio/Baseline
- **Descripción**: Este algoritmo predice las calificaciones de los usuarios a los ítems basándose en una distribución aleatoria que intenta imitar la distribución general de las calificaciones en el conjunto de datos. Es más un modelo de línea base que un enfoque predictivo sofisticado. No utiliza factorización de matrices ni similitudes entre usuarios o ítems.
- **Uso**: Principalmente para establecer un punto de comparación básico. No se considera un enfoque tradicional de filtrado colaborativo.

In [53]:
algo = NormalPredictor()
cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3178  1.3164  1.3166  1.3160  1.3112  1.3156  0.0023  
MAE (testset)     1.0215  1.0161  1.0186  1.0190  1.0111  1.0173  0.0035  
Fit time          0.06    0.08    0.09    0.10    0.08    0.08    0.01    
Test time         0.07    0.17    0.09    0.08    0.18    0.12    0.05    


{'test_rmse': array([1.31784513, 1.31642592, 1.31661414, 1.31603454, 1.31124094]),
 'test_mae': array([1.02153313, 1.01611645, 1.01862797, 1.01897689, 1.01110487]),
 'fit_time': (0.06013655662536621,
  0.08403348922729492,
  0.08940362930297852,
  0.10064435005187988,
  0.08417344093322754),
 'test_time': (0.06819486618041992,
  0.1725163459777832,
  0.08813691139221191,
  0.08057403564453125,
  0.176100492477417)}

## 3. **KNNBasic**
- **Enfoque**: Basado en Memoria
- **Descripción**: Este algoritmo es un ejemplo de filtrado colaborativo basado en memoria, específicamente, un enfoque basado en ítems cuando `user_based: False` (como en tu ejemplo) o basado en usuarios si `user_based: True`. Calcula las similitudes entre ítems (o usuarios) para hacer recomendaciones, basándose en las k calificaciones más similares.
- **Uso**: Para recomendaciones que dependen directamente de la similitud entre los ítems o entre los usuarios, sin la necesidad de construir un modelo predictivo subyacente.


In [54]:
# algo = KNNBasic(sim_options={'user_based': False} , k=20) # https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measure-configuration
# cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

## 2. **NMF (Non-negative Matrix Factorization)**
- **Enfoque**: Factorización de Matrices
- **Descripción**: Similar al SVD, NMF es una técnica de factorización de matrices que descompone la matriz de utilidad en dos matrices de factores no negativos, representando las características latentes de los usuarios e ítems, respectivamente. A diferencia de SVD, NMF impone la restricción de que todos los elementos de las matrices factorizadas sean no negativos, lo cual puede resultar en una interpretación más intuitiva de los factores latentes.
- **Uso**: Para hacer predicciones de calificaciones y recomendaciones basándose en características latentes descubiertas a través de la factorización.

In [55]:
algo = NMF()
cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0083  0.9980  0.9953  1.0135  0.9919  1.0014  0.0082  
MAE (testset)     0.7567  0.7372  0.7414  0.7546  0.7409  0.7462  0.0079  
Fit time          2.15    2.37    2.19    2.28    2.03    2.20    0.12    
Test time         0.08    0.25    0.06    0.08    0.08    0.11    0.07    


{'test_rmse': array([1.008343  , 0.99803668, 0.99525883, 1.0134684 , 0.99191326]),
 'test_mae': array([0.75672691, 0.73718135, 0.74141197, 0.75456219, 0.74091612]),
 'fit_time': (2.1514668464660645,
  2.3652443885803223,
  2.185018539428711,
  2.2819364070892334,
  2.025695323944092),
 'test_time': (0.07661032676696777,
  0.2505931854248047,
  0.06391406059265137,
  0.08066821098327637,
  0.08402848243713379)}

## Filtrado colaborativo basado en memoria

Se va a realizar un filtrado colaborativo, específicamente un **filtrado colaborativo basado en memoria** utilizando un **enfoque basado en ítems**. La implementación usa el algoritmo de **k-Nearest Neighbors (k-NN)** para encontrar las películas más similares a una dada.

### Detalles del Proceso:

1. **Creación de la Matriz de Utilidad**: Se transforma el conjunto de datos en una matriz (pivot_table) donde las filas representan ítems (películas) y las columnas usuarios, con las calificaciones como valores. Los valores faltantes se llenan con cero. Esto es típico de un enfoque basado en memoria.

2. **Uso de CSR Matrix**: La conversión de la matriz de utilidad a una **CSR Matrix** (Compressed Sparse Row) optimiza el almacenamiento y las operaciones matemáticas para matrices dispersas, lo cual es común en sistemas de recomendación debido a la gran cantidad de calificaciones faltantes.

3. **Aplicación de k-NN**: Se utiliza el algoritmo k-NN para calcular las distancias (similitudes) entre las películas basándose en las calificaciones de los usuarios. Se configura para usar la métrica de similitud del coseno, lo cual es adecuado para medir similitudes en espacios de alta dimensionalidad como es el caso de las matrices de utilidad en sistemas de recomendación.

4. **Recomendaciones Basadas en Similitud**: Al buscar las películas más cercanas (similares) a una dada, se están generando recomendaciones basadas directamente en la similitud de patrones de calificación entre ítems. Esto es característico de un enfoque basado en ítems dentro del filtrado colaborativo basado en memoria.

### Enfoque: Basado en Ítems

El uso de k-NN con una matriz donde las películas son las entidades sobre las cuales se calculan las similitudes, y la selección de vecinos más cercanos basada en estas similitudes, clasifica este método dentro del **filtrado colaborativo basado en memoria** y, más específicamente, en un **enfoque basado en ítems**. En este enfoque, las recomendaciones se generan buscando ítems similares a aquellos que el usuario ya ha calificado positivamente, asumiendo que si a un usuario le gustó un ítem, también le gustarán otros ítems similares.

La forma en que se organiza la tabla pivote afecta directamente a cómo se calculan las similitudes y, por lo tanto, cómo se realizan las recomendaciones. La elección de la orientación de la tabla pivote influye en cómo se consideran las relaciones entre usuarios y películas. Aquí hay algunas consideraciones:

1. **Recomendaciones basadas en usuarios:**
   - Con `df.pivot_table(index='userId', columns='title', values='rating')`, las similitudes entre usuarios se calculan en función de sus calificaciones para las mismas películas.
   - Las recomendaciones se generan comparando las preferencias de un usuario con las de otros usuarios que han calificado de manera similar las películas que el usuario ha visto.

2. **Recomendaciones basadas en películas:**
   - Con `df.pivot_table(index='title', columns='userId', values='rating')`, las similitudes entre películas se calculan en función de cómo han sido calificadas por los mismos usuarios.
   - Las recomendaciones se generan comparando las características (calificaciones) de una película con las de otras películas que han sido calificadas de manera similar por los usuarios.

En resumen, la elección de la orientación de la tabla pivote afecta a la dirección de las comparaciones de similitud. Puedes elegir la orientación que mejor se adapte a tus objetivos específicos y a la lógica de tu sistema de recomendación. En muchos casos, se prefiere organizar la tabla pivote de manera que las filas representen usuarios, ya que las recomendaciones suelen centrarse en usuarios similares.

In [56]:
# RESET dataframe variable
df = dataFrame_sorted_by_timestamp.copy()
df.head()

,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
53748,73725,190401,"Scotland, coverup, netflix, dark, Netflix orig...","original, mentor, catastrophe, silly fun, grea...",Calibre (2018),Thriller,1574300676,3.0
30003,122409,1500,"hitman, rekindled love, romance, predictable, ...","comedy, hit men, hitman, off-beat comedy, assa...",Grosse Pointe Blank (1997),Comedy|Crime|Romance,1574276946,2.5
26209,160473,47,"Sloth, s.w.a.t., very good, Atmospheric, drug ...","powerful ending, police investigation, great e...",Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1574238626,4.5
19356,42965,79132,"menswear - outstanding, Orriginal screenplay, ...","complex, dreams, complicated, visually appeali...",Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,1574238008,5.0
38586,96399,2527,"AI, wissenschaftliche sci-fi, cyborgs, chase, ...","futuristic, robot, androids, future, robots",Westworld (1973),Action|Sci-Fi|Thriller|Western,1574219255,2.5


In [57]:
pivot_table_result_based_film = df.pivot_table(index='movieId', columns='userId', values='rating')
pivot_table_result_based_film.fillna(0, inplace=True)
pivot_table_result_based_film.shape
pivot_table_result_based_film.head()
data_final = pivot_table_result_based_film.copy()

In [58]:
def get_movie_recommendation(movie_name, movies_data, csr_matrix):
    """
    Recomienda películas similares basadas en el nombre de una película utilizando k-NN.

    Parámetros:
    - movie_name: Nombre de la película para la cual se desean recomendaciones.
    - movies_data: DataFrame con información sobre películas.

    Retorna:
    - DataFrame con títulos de películas similares y sus distancias.
    """

    # Número de recomendaciones a obtener
    n = 10

    # Filtrar películas que coinciden con el nombre proporcionado
    movie_list = movies_data[movies_data['title'].str.contains(movie_name, regex=False)]

    if len(movie_list):
        # Obtener el índice de la película (movieId)
        movie_idx = movie_list.iloc[0]['movieId']

        # Obtener el índice de la película en el conjunto de datos final
        movie_idx = data_final[data_final['movieId'] == movie_idx].index[0]

        knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
        knn.fit(csr_matrix)

        # Calcular distancias y obtener índices de las películas más similares usando k-NN
        distances, indices = knn.kneighbors(csr_matrix[movie_idx], n_neighbors=n + 1)

        # Organizar y obtener índices de películas recomendadas
        rec_movie_indices = sorted(list(zip(indices.squeeze(), distances.squeeze())), key=lambda x: x[1])[1::1]

        # Inicializar listas para almacenar títulos y distancias recomendadas
        recommend = []
        recommend2 = []

        # Obtener títulos y distancias de las películas recomendadas
        for val in rec_movie_indices:
            movie_idx = data_final.iloc[val[0]]['movieId']
            idx = movies_data[movies_data['movieId'] == movie_idx].index
            recommend.append(movies_data.iloc[idx]['title'].values[0])
            recommend2.append(val[1])

        # Crear un DataFrame con títulos y distancias
        df1 = pd.DataFrame(recommend)
        df2 = pd.DataFrame(recommend2)
        df = pd.concat([df1, df2], axis='columns')
        df.columns = ['Title', 'Distance']
        df.set_index('Distance', inplace=True)

        return df
    else:
        return "No movies found. Please check your input"


In [59]:
csr_data = csr_matrix(data_final.values)
data_final.reset_index(inplace=True)

In [60]:
get_movie_recommendation('Harry Potter and the Goblet of Fire (2005)', df[['movieId', 'title', 'genres']], csr_data)

,Title
Distance,
0.399650,"Prince of Egypt, The (1998)"
0.442960,Phone Booth (2002)
0.446513,K-PAX (2001)
0.490317,"Bourne Supremacy, The (2004)"
0.582144,Victoria (2015)
0.587211,Scarface (1983)
0.715062,Alpha (2018)
0.732694,Casino (1995)
0.755215,Spirited Away (Sen to Chihiro no kamikakushi) ...


## FILTRADO COLABORATIVO BASADO EN MEMORIA CON SIMILITUD DEL COSENO

Un filtrado colaborativo basado en memoria utiliza directamente las calificaciones de los usuarios para hacer recomendaciones, sin construir un modelo subyacente. Hay dos tipos principales: basado en usuarios y basado en ítems. A continuación, implementaré un ejemplo sencillo de un sistema de recomendación basado en usuarios utilizando Python y pandas. Este enfoque recomienda ítems al usuario objetivo basándose en las calificaciones de usuarios similares.

```python
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recommend_movies_user_based(user_id, ratings_matrix, n_recommendations=5):
    """
    Recomienda películas para un usuario utilizando el enfoque de filtrado colaborativo basado en usuarios.
    
    Args:
    - user_id: ID del usuario para el cual se quieren las recomendaciones.
    - ratings_matrix: DataFrame de pandas con usuarios en filas, películas en columnas y calificaciones como valores.
    - n_recommendations: Número de recomendaciones a retornar.
    
    Returns:
    - Lista de títulos de películas recomendadas.
    """
    
    # Calcular la similitud coseno entre los usuarios
    user_similarity = cosine_similarity(ratings_matrix)
    user_similarity = pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)
    
    # Obtener las calificaciones del usuario objetivo y eliminarlas de las calificaciones a considerar
    user_ratings = ratings_matrix.loc[user_id, :]
    other_users = ratings_matrix.drop(user_id)
    
    # Calcular los pesos (similitudes) para cada usuario respecto al usuario objetivo
    similarities = user_similarity[user_id].drop(user_id)
    
    # Calcular el puntaje ponderado para cada película basado en las similitudes y las calificaciones de los otros usuarios
    weighted_ratings = np.dot(user_similarity.drop(user_id, axis=1).values, other_users.fillna(0).values)
    sum_of_weights = np.abs(user_similarity.drop(user_id, axis=1).values).sum(axis=1)
    
    # Evitar la división por cero
    sum_of_weights[sum_of_weights == 0] = 1
    
    # Calcular el puntaje promedio ponderado para cada película
    weighted_average_ratings = weighted_ratings[user_id - 1, :] / sum_of_weights[user_id - 1]
    
    # Convertir a Series para facilitar el manejo
    weighted_average_ratings = pd.Series(weighted_average_ratings, index=ratings_matrix.columns)
    
    # Filtrar las películas que el usuario ya ha calificado
    weighted_average_ratings = weighted_average_ratings[user_ratings[user_ratings.isna()].index]
    
    # Retornar las n películas con el puntaje promedio ponderado más alto
    return weighted_average_ratings.nlargest(n_recommendations).index.tolist()
```

Para utilizar esta función, necesitas tener un DataFrame `ratings_matrix` donde las filas representen a los usuarios, las columnas representen a las películas, y los valores sean las calificaciones que los usuarios han dado a las películas. Aquí se asume que `user_id` es un índice válido en `ratings_matrix` y que las películas no calificadas por el usuario están representadas por `NaN`.

Este enfoque se basa en la similitud entre los usuarios para hacer recomendaciones, asumiendo que los usuarios con gustos similares calificarán las películas de manera similar.

## FILTRADO COLABORATIVO BASADO EN MEMORIA CON LA CORRELACIÓN DE PEARSON

Para crear un sistema de recomendación basado en memoria sin utilizar la similitud del coseno, podemos emplear la correlación de Pearson como medida de similitud. La correlación de Pearson mide la correlación lineal entre dos variables, en este caso, las calificaciones entre usuarios. Aquí te muestro cómo implementar un sistema de recomendación basado en usuarios utilizando la correlación de Pearson para calcular la similitud entre usuarios:

```python
import pandas as pd
import numpy as np

def recommend_movies_pearson(user_id, ratings_matrix, n_recommendations=5):
    """
    Recomienda películas para un usuario utilizando el enfoque de filtrado colaborativo basado en usuarios
    y la correlación de Pearson para calcular la similitud entre usuarios.
    
    Args:
    - user_id: ID del usuario para el cual se quieren las recomendaciones.
    - ratings_matrix: DataFrame de pandas con usuarios en filas, películas en columnas y calificaciones como valores.
    - n_recommendations: Número de recomendaciones a retornar.
    
    Returns:
    - Lista de títulos de películas recomendadas.
    """
    
    # Calcular la correlación de Pearson entre todos los usuarios
    user_similarity = ratings_matrix.T.corr(method='pearson')
    
    # Obtener las calificaciones del usuario objetivo
    user_ratings = ratings_matrix.loc[user_id, :].dropna()
    
    # Inicializar un diccionario para guardar los puntajes ponderados
    scores = {}
    
    for movie_id in ratings_matrix.columns:
        # Saltar películas que el usuario ya ha calificado
        if pd.notna(ratings_matrix.loc[user_id, movie_id]):
            continue
        
        # Calcular el puntaje ponderado para la película
        total_similarity = 0
        weighted_score = 0
        
        for other_user in ratings_matrix.index:
            if user_id != other_user and pd.notna(ratings_matrix.loc[other_user, movie_id]):
                # Obtener la similitud entre el usuario objetivo y otro usuario
                similarity = user_similarity.loc[user_id, other_user]
                
                # Sumar a la puntuación ponderada
                weighted_score += similarity * ratings_matrix.loc[other_user, movie_id]
                
                # Sumar la similitud al total para normalización
                total_similarity += abs(similarity)
        
        # Calcular el puntaje final normalizado si hay similitudes totales
        if total_similarity > 0:
            scores[movie_id] = weighted_score / total_similarity
    
    # Ordenar las películas por su puntuación ponderada y obtener las mejores n recomendaciones
    recommended_movies = sorted(scores, key=scores.get, reverse=True)[:n_recommendations]
    
    return recommended_movies
```

Este código asume que `ratings_matrix` es un DataFrame de pandas donde las filas son los `userId` y las columnas son las `movieId` con las calificaciones como valores. Las películas no calificadas por un usuario deben tener valores `NaN`.

La principal diferencia con el enfoque anterior es el uso de la correlación de Pearson en lugar de la similitud del coseno para calcular la similitud entre usuarios. La correlación de Pearson considera tanto las calificaciones promedio de los usuarios como su tendencia a calificar de manera similar, proporcionando un enfoque ligeramente distinto para identificar usuarios similares.